In [1]:

import numpy as np
import tensorflow as tf
import keras
from keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline

from keras.optimizers import Adam

from fully_conv_model_for_lidar import fcn_model
from util_func import *
from train_2 import *

from keras.utils.generic_utils import get_custom_objects
#loss = SSD_Loss(neg_pos_ratio=neg_pos_ratio, alpha=alpha)
get_custom_objects().update({"my_loss": my_loss})

import mayavi.mlab


Using TensorFlow backend.


# Load model

In [2]:

model = load_model('saved_model/cs_model_May_29_epoch_180.h5')


[<tf.Tensor 'gradients/split_1_grad/concat:0' shape=(?, 64, 256, 8) dtype=float32>, None, None]


# Test prediction

In [3]:

lidar = np.load('test_data/19_f2/lidar_239.npy')
print(len(lidar))
all_boxes, cluster_boxes = predict_boxes(model, lidar, cluster = True,
                                         seg_thres=0.5, cluster_dist = 0.7, min_dist = 1.5, neigbor_thres = 3)
                                         #seg_thres=0.5, cluster_dist = 0.7, min_dist = 1.5, neigbor_thres = 3)
print(len(all_boxes))
print(len(cluster_boxes))


30858
79
2


# Visualize lidar points and bounding box, with ground truth tracklet (optional)

In [4]:

def viz_point_box_groundtruth(points, pred_boxes, gt_boxes, vals="distance"):
    x = points[:, 0]  # x position of point
    y = points[:, 1]  # y position of point
    z = points[:, 2]  # z position of pointfrom mpl_toolkits.mplot3d import Axes3D
    # r = lidar[:, 3]  # reflectance value of point
    d = np.sqrt(x ** 2 + y ** 2)  # Map Distance from sensor

    # Plot using mayavi -Much faster and smoother than matplotlib
    #import mayavi.mlab
    if vals == "height":
        col = z
    else:
        col = d

    fig = mayavi.mlab.figure(bgcolor=(0, 0, 0), size=(640, 360))
    mayavi.mlab.points3d(x, y, z,
                         col,          # Values used for Color
                         mode="point",
                         colormap='spectral', # 'bone', 'copper', 'gnuplot'
                         # color=(0, 1, 0),   # Used a fixed (r,g,b) instead
                         figure=fig,
                         )
    
    # predicted boxes
    pred_color = (1,1,1)
    for i in range(len(pred_boxes)):
        car = pred_boxes[i]
        x = car[:,0]
        y = car[:,1]
        z = car[:,2]

        mayavi.mlab.plot3d(x[:4], y[:4], z[:4], tube_radius=0.025, color=pred_color)#, colormap='Spectral')
        mayavi.mlab.plot3d(x[[0,3]], y[[0,3]], z[[0,3]], tube_radius=0.025, color=pred_color)
        mayavi.mlab.plot3d(x[[0,4]], y[[0,4]], z[[0,4]], tube_radius=0.025, color=pred_color)
        mayavi.mlab.plot3d(x[[1,5]], y[[1,5]], z[[1,5]], tube_radius=0.025, color=pred_color)
        mayavi.mlab.plot3d(x[[2,6]], y[[2,6]], z[[2,6]], tube_radius=0.025, color=pred_color)
        mayavi.mlab.plot3d(x[[3,7]], y[[3,7]], z[[3,7]], tube_radius=0.025, color=pred_color)
        
        mayavi.mlab.plot3d(x[-4:], y[-4:], z[-4:], tube_radius=0.025, color=pred_color)#, colormap='Spectral')
        mayavi.mlab.plot3d(x[[4,7]], y[[4,7]], z[[4,7]], tube_radius=0.025, color=pred_color)
    
    #ground truth boxes
    gt_color = (1,1,0)
    for i in range(len(gt_boxes)):
        car = gt_boxes[i]
        x = car[:,0]
        y = car[:,1]
        z = car[:,2]

        mayavi.mlab.plot3d(x[:4], y[:4], z[:4], tube_radius=0.025, color=gt_color)#, colormap='Spectral')
        mayavi.mlab.plot3d(x[[0,3]], y[[0,3]], z[[0,3]], tube_radius=0.025, color=gt_color)
        mayavi.mlab.plot3d(x[[0,4]], y[[0,4]], z[[0,4]], tube_radius=0.025, color=gt_color)
        mayavi.mlab.plot3d(x[[1,5]], y[[1,5]], z[[1,5]], tube_radius=0.025, color=gt_color)
        mayavi.mlab.plot3d(x[[2,6]], y[[2,6]], z[[2,6]], tube_radius=0.025, color=gt_color)
        mayavi.mlab.plot3d(x[[3,7]], y[[3,7]], z[[3,7]], tube_radius=0.025, color=gt_color)
        
        mayavi.mlab.plot3d(x[-4:], y[-4:], z[-4:], tube_radius=0.025, color=gt_color)#, colormap='Spectral')
        mayavi.mlab.plot3d(x[[4,7]], y[[4,7]], z[[4,7]], tube_radius=0.025, color=gt_color)
        
    mayavi.mlab.show()


In [5]:

# get bounding box from udacity tracklet
ground_truth_box = tracklet_gt_to_box('./one_frame_data/tracklet_gt.xml', tracklet_idx=0, frame_number=1000)
print ground_truth_box


Parsing Tracklet file ./one_frame_data/tracklet_gt.xml
File contains 1 Tracklets
Loaded 1 Tracklets
[[ 8.203899  0.761408 -2.590839]
 [ 8.203899 -0.686392 -2.590839]
 [ 9.778699 -0.686392 -2.590839]
 [ 9.778699  0.761408 -2.590839]
 [ 8.203899  0.761408  1.650961]
 [ 8.203899 -0.686392  1.650961]
 [ 9.778699 -0.686392  1.650961]
 [ 9.778699  0.761408  1.650961]]


In [6]:

viz_point_box_groundtruth(points=lidar, pred_boxes=cluster_boxes, gt_boxes=[])


# Generate tracklet file (Predict boxes from .npy files and save to .xml file)

In [7]:

generate_tracklet(pred_model=model, input_folder='test_data/19_f2', output_file='test_data/round_1_tracklet.xml', \
                 cluster=True, seg_thres=0.5, cluster_dist=0.7, min_dist=1.5, neigbor_thres=3)


frame 0: 0 boxes detected
frame 1: 0 boxes detected
frame 2: 0 boxes detected
frame 3: 0 boxes detected
frame 4: 0 boxes detected
frame 5: 0 boxes detected
frame 6: 0 boxes detected
frame 7: 0 boxes detected
frame 8: 0 boxes detected
frame 9: 0 boxes detected
frame 10: 0 boxes detected
frame 11: 0 boxes detected
frame 12: 0 boxes detected
frame 13: 0 boxes detected
frame 14: 0 boxes detected
frame 15: 0 boxes detected
frame 16: 0 boxes detected
frame 17: 0 boxes detected
frame 18: 0 boxes detected
frame 19: 0 boxes detected
frame 20: 0 boxes detected
frame 21: 0 boxes detected
frame 22: 1 boxes detected
frame 23: 1 boxes detected
frame 24: 0 boxes detected
frame 25: 0 boxes detected
frame 26: 0 boxes detected
frame 27: 0 boxes detected
frame 28: 0 boxes detected
frame 29: 0 boxes detected
frame 30: 0 boxes detected
frame 31: 0 boxes detected
frame 32: 1 boxes detected
frame 33: 1 boxes detected
frame 34: 2 boxes detected
frame 35: 2 boxes detected
frame 36: 2 boxes detected
frame 37: 2